In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

In [7]:
def search(artist_list, checkpoint_interval=1000):
    artist_relations = {}
    searched_artist_names = {}
    not_exist_artist = []
    for i, a in tqdm(enumerate(artist_list), total=len(artist_list), position=0, desc="Processing artists"):

        aa = a.replace(" ", "+")
        url = f'https://inflooenz.com/?artist={aa}&submit=Search'
        try:
            page = requests.get(url)
            page.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error: {a} not found")
            continue

        soup = BeautifulSoup(page.content, 'html.parser')

        notexist= soup.find('h3', attrs={'class':'empty'})
        if notexist:
            print(f'{a} is not in the database')
            not_exist_artist.append(a)
  


        # Extract the searched artist name
        artist_name_tag = soup.find('h1')
        searched_artist_name = None
        if artist_name_tag:
            searched_artist_name_tag = artist_name_tag.find('span', {'class': 'artist', 'itemprop': 'name'})
            if searched_artist_name_tag:
                searched_artist_name = searched_artist_name_tag.text

        followers = []
        influencers = []

        followers_list = soup.find('ul', attrs={'class':'influences-list', 'id':'followers-list'})
        if followers_list:
            for follower in followers_list.find_all('li')[:-1]:
                if follower.text != "":
                    followers.append(follower.text)
        influencers_list = soup.find('ul', attrs={'class':'influences-list', 'id':'influencers-list'})
        if influencers_list:
            for influencer in influencers_list.find_all('li')[:-1]:
                if influencer.text != "":
                    influencers.append(influencer.text)

        if searched_artist_name:
            artist_relations[a] = {'followers': followers, 'influencers': influencers}
            searched_artist_names[a] = searched_artist_name
        else:
            artist_relations[a] = {'followers': followers, 'influencers': influencers}
            searched_artist_names[a] = a


    return artist_relations, searched_artist_names, not_exist_artist

In [8]:
a7x = search(['Avenged Sevenfold', 'Beasts Of Bourbon'])
a7x

Processing artists: 100%|██████████| 2/2 [00:08<00:00,  4.38s/it]

Beasts Of Bourbon is not in the database


({'Avenged Sevenfold': {'followers': ['Asking Alexandria',
    'Black Veil Brides',
    'Andy Black',
    'Machine Gun Kelly'],
   'influencers': ['Alice in Chains',
    'Rage Against the Machine',
    'Iron Maiden',
    "Guns N' Roses",
    'Bad Religion',
    'Dream Theater',
    'Pantera',
    'Metallica',
    'AFI',
    'Black Flag',
    'Corrosion of Conformity',
    'Deftones',
    'Korn',
    'Misfits',
    'Slayer',
    'The Vandals']},
  'Beasts Of Bourbon': {'followers': [], 'influencers': []}},
 {'Avenged Sevenfold': 'Avenged Sevenfold',
  'Beasts Of Bourbon': 'Beasts Of Bourbon'},
 ['Beasts Of Bourbon'])